## How To Perform an Amplitude Rabi Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to find the optimal pulse amplidue to induce a $\pi$-rotation for a given qubit transition using the `amplitude_rabi` experiment included in the LabOne Q Applications Library. In an amplitude-Rabi experiment, you'll vary the drive-pulse amplitude, $A$, and measure the state of the qubit as it rotates around the Bloch sphere depending on $A$, as depicted below. 

![](../../../images/rabi_pulses_and_sphere.svg "Demonstration of pulses and the corresponding rotation around the Bloch sphere during an amplitude Rabi experiment.")

For a qubit oscialling between the ground state, $\ket{0}$, and excited state, $\ket{1}$, the population behaves depends on the drive-pulse amplitude according to:

$$\rho(\ket{1}) = sin^2 \bigg(\int_{0}^{\tau}\frac{\Omega_{R}(A)}{2}dt \bigg) = sin^2\bigg(\frac{\theta}{2}\bigg)$$

This results in data that will, ignoring any other effects, look like:

![](<../../../images/rabi_plot.svg> "Plot of Rabi oscillations with the population of the excited state on the left axis and the pulse amplitude on the bottom axis. Measurement points are depicted as cyan circles.")

### Imports

You'll start by importing the amplitude-Rabi experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo quantum platform containing a qpu and device setup to run in emulation mode. See the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) for more details about the quantum platform and how to create your experimental setup.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import amplitude_rabi
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Let's first create the options class for the amplitude-Rabi experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = amplitude_rabi.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the experiment is run on the qubit $|g\rangle \leftrightarrow |e\rangle$ (`transition`)
- calibration traces are added (`use_cal_traces=True`) for the states $|g\rangle$ and $|e\rangle$ (`cal_states`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel.

Note that the fit fails in emulation mode and the $\pi$- and $\pi/2$-pulse amplitudes cannot be extracted.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplitudes=[np.linspace(0, 1, 11), np.linspace(0, 0.5, 11)],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(compiled_experiment, length=50e-6)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `amplitude_rabi` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the Quantum Operations tutorial (add link).

In [ ]:
amplitude_rabi.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow. Because the $\pi$ and $\pi/2$ amplitudes cannot be extracted in emulation mode, the new qubit parameters are empty.

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

Great! You've now run your Rabi experiment. Check out other experiments in this manual to keep characterizing your qubits.